In [ ]:
from openai import OpenAI

client = OpenAI()
vector_store = client.vector_stores.create(
    name="EOI_Documents_VS",
)
client.vector_stores.list()


In [ ]:
import os
from openai import OpenAI
from pydantic import BaseModel
from typing import List, Optional
from dotenv import load_dotenv

load_dotenv()

class PurchaserModel(BaseModel):
    First_Name: str
    Last_Name: str
    Purchaser_Email: str
    Purchaser_Mobile: str


class EOIExtractedModel(BaseModel):
    Purchaser: List[PurchaserModel]
    Residential_Address: str
    Lot_Number: str
    Property_Address: str
    Project_Name: str
    Total_Price: str
    Land_Price: str
    Build_Price: str
    Finance_Terms: str
    Solicitor_Name: str
    Solicitor_Email: str
    Finance_Provider: Optional[str] = None


def upload_file_to_openai(client, file_path: str):
    with open(file_path, "rb") as f:
        uploaded = client.files.create(
            file=f,
            purpose="assistants"   # required for responses API
        )
    return uploaded.id


def extract_structured_fields(email_body, pdf_path: str):
    client = OpenAI()

    # 1️⃣ Upload file first → get file_id
    file_id = upload_file_to_openai(client, pdf_path)

    PROMPT_TEMPLATE = """
    You are an expert document extraction AI. You will be given:

    1. An Expression of Interest (EOI) PDF.
    2. The full email body that accompanied the EOI PDF:
    ---
    {email_text}
    ---

    Your job:

    Extract ONLY the following fields and output **VALID JSON** in EXACTLY this structure:

    {{
    "Purchaser":[
        {{
            "First_Name": "<str>",
            "Last_Name": "<str>",
            "Purchaser_Email": "<str>",
            "Purchaser_Mobile": "<str>"
        }}
    ],
    "Residential_Address": "<str>",
    "Lot_Number": "<str>",
    "Property_Address": "<str>",
    "Project_Name": "<str>",
    "Total_Price": "<str>",
    "Land_Price": "<str>",
    "Build_Price": "<str>",
    "Finance_Terms": "<str>",
    "Solicitor_Name": "<str>",
    "Solicitor_Email": "<str>",
    "Finance_Provider": "<str or null>"
    }}

    Rules:

    1. Extract fields **from the PDF first**.
    2. If a field is missing in the PDF, search for it in the accompanying email text.
    3. If still missing, return an empty string (""), EXCEPT:
    - Finance_Provider → return null when not available.
    4. DO NOT add any extra keys.
    5. Purchaser should be a list of objects, even if there is only one.
    6. Return only the JSON. No explanation or markdown.
    """

    prompt = PROMPT_TEMPLATE.format(email_text=email_body)

    # 2️⃣ Reference the file via file_id in the input
    response = client.responses.parse(
        model="gpt-4.1",
        input=[
            {
                "role": "user",
                "content": [
                    {"type": "input_text", "text": prompt},
                    {"type": "input_file", "file_id": file_id}
                ]
            }
        ],
        text_format=EOIExtractedModel  # Pydantic automatic validation!
    )
    ingest_eoi_to_vector_store(response.output_text)

    return True

def ingest_eoi_to_vector_store(EOI_JSON):
    client = OpenAI()
    vector_store_id = os.getenv("OPENAI_VS_ID")
    with open("temp.txt", "w", encoding="utf-8") as f:
        f.write(EOI_JSON)
    response = client.vector_stores.files.upload_and_poll(vector_store_id=vector_store_id,
                                                        file=open("temp.txt", "rb"),
                                                        poll_interval_ms=2000)
    os.remove("temp.txt")
    return response

# Run
# if __name__ == "__main__":
email_body = """
From: sales@onecorpaustralia.com.au
To: support@onecorpaustralia.com.au
Subject: EOI Signed:5 Drive Abu, NSW, 2100

Body:
Hi team,

The clients Talky Bab has signed the Expression of Interest for:

5 Drive Abu, NSW, 2100
Total Price: $500,000

EOI document is attached. Please proceed with the contract workflow.

Thanks,
Adrian
OneCorp
"""
out = extract_structured_fields(email_body, "data/EOI_Talk.pdf")


In [3]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv


def search_vector_store(email: str):
    load_dotenv()
    vector_store_id      = os.getenv("OPENAI_VS_ID")
    client = OpenAI()

    query = """
    Extract the purchaser(s) name and property address from the following email regarding the Contract of Sale:
    ```
    {email}
    ```
    In the following Format:
    Purchaser(s) Name: <Full Name(s)>
    Property Address: <Full Address>
    """
    response = client.chat.completions.create(
        model="gpt-4.1-mini",   # or your preferred model
        messages=[
            {"role": "user", "content": query.format(email=email)}
        ],
        temperature=0.2
    )

    results = client.vector_stores.search(
        vector_store_id=vector_store_id,
        query=response.choices[0].message.content,
        max_num_results=1)

    return json.loads(results.data[0].content[0].text)

In [26]:
import json
from openai import OpenAI
from typing import Optional, List
from pydantic import BaseModel


class IncorrectField(BaseModel):
    Field: str
    EOI_Value: Optional[str]  # Some fields may be empty in the EOI
    Contract_Value: Optional[str]  # null allowed when missing from contract


class ContractValidationModel(BaseModel):
    Contract_Validation: bool
    Incorrect_Fields: List[IncorrectField]

def upload_file_to_openai(client, file_path: str):
    with open(file_path, "rb") as f:
        uploaded = client.files.create(
            file=f,
            purpose="assistants"   # required for responses API
        )
    return uploaded.id
def contract_validation_agent(email_body, pdf_path: str):
    client = OpenAI()
    # 1️⃣ Upload file first → get file_id
    file_id = upload_file_to_openai(client, pdf_path)

    CONTRACT_CHECKER_PROMPT = """
You are a Contract Validation AI Agent working for OneCorp Australia. Your job is to compare Contract of Sale values against the correct values extracted from an Expression of Interest (EOI).

The EOI JSON below contains authoritative values:
---
{eoi_json}
---

Important Validation Rules (follow EXACTLY):

1. A mismatch MUST be reported **only if BOTH of the following are true**:
    a. The EOI field has a real, non-empty value  
       (meaning: NOT null, NOT "", NOT " ", NOT missing)  
    b. The Contract contains a value for the same field AND it conflicts with the EOI value.

2. If the EOI value is empty (null, "", or whitespace), then:
    - EVEN IF the contract contains a value → **IGNORE this field completely**  
    - It must NOT be included in Incorrect_Fields.

3. If the contract does not mention a field at all:
    - DO NOT count it as a mismatch (missing contract fields are allowed).

4. Subtle contradictions MUST still be flagged.
   Example:
   EOI Finance_Terms = "Not Subject to Finance"
   Contract clause implies responsibility for securing finance approval.
   → Treat this as a mismatch in Finance_Terms.

5. Only detect mismatches for fields appearing in BOTH:
    - EOI has a real value
    - Contract explicitly states a different value

6. NEVER output mismatches where:
    - EOI value is empty/null  
    - EOI value is missing  
    - Contract has additional details not present in EOI  

### Extract from the contract:
- Purchaser Names
- Purchaser Emails
- Purchaser Mobiles
- Residential Address (if present)
- Lot Number
- Property Address
- Project Name
- Total Price
- Land Price
- Build Price
- Finance Terms
- Solicitor Name
- Solicitor Email
- Finance Provider (optional)

Then apply the rules above.

### OUTPUT FORMAT (strict)

Return ONLY this JSON:

{{
  "Contract_Validation": <true_or_false>,
  "Incorrect_Fields": [
      {{
        "Field": "<Field_Name>",
        "EOI_Value": "<Value_From_EOI>",
        "Contract_Value": "<Value_From_Contract>"
      }}
  ]
}}

Rules:
- If **no mismatches** exist → return:
{{
  "Contract_Validation": true,
  "Incorrect_Fields": []
}}

- Otherwise:
{{
  "Contract_Validation": false,
  "Incorrect_Fields": [...]
}}

- Do NOT include fields where EOI_Value was empty/null.
- Do NOT include fields missing in the contract.
- No explanations, no markdown, no comments.
    """


    eoi_json = search_vector_store(email_body)
    prompt = CONTRACT_CHECKER_PROMPT.format(eoi_json=eoi_json)

    # 2️⃣ Reference the file via file_id in the input
    response = client.responses.parse(
        model="gpt-4.1",
        input=[
            {
                "role": "user",
                "content": [
                    {"type": "input_text", "text": prompt},
                    {"type": "input_file", "file_id": file_id}
                ]
            }
        ],
        text_format=ContractValidationModel  # Pydantic automatic validation!
    )

    response = json.loads(response.output_text)

    output = {}
    purchasers = ""
    for i in eoi_json["Purchaser"]:
        purchasers += f"{i['First_Name']} {i['Last_Name']} & "
    purchasers = purchasers.rstrip(" & ")
    address = eoi_json["Property_Address"]

    if response["Contract_Validation"]:
        # Generate solicitor email
        output["Contract_Validation"] = True
        output["solicitor_email"] = {}
        output["solicitor_email"]["Subject"] = f"Contract of Sale Validated Successfully for {purchasers} - {address}"
        output["solicitor_email"]["Body"] = f"""
Hi {eoi_json['Solicitor_Name']},

Please find attached the contract for {purchasers} for your review.

Let us know if you have any questions.

Kind regards,
OneCorp"""
        output["solicitor_email"]["To"] = eoi_json["Solicitor_Email"]

    else:
        # Construct revision email to Vendor and internal team
        incorrect_fields = ""
        for field in response["Incorrect_Fields"]:
            incorrect_fields += f"- {field['Field']}: EOI Value = '{field['EOI_Value']}', Contract Value ='{field['Contract_Value']}'\n"
        output["Contract_Validation"] = False
        output["Revision_Email"] = {}
        output["Revision_Email"]["Subject"] = f"Contract of Sale Discrepancies for {purchasers} - {address}"
        output["Revision_Email"]["Body"] = f"""
Greetings,
We have reviewed the Contract of Sale for {purchasers} regarding the property at {address}. During validation, we identified the following discrepancies compared to the purchasers’ Expression of Interest: \n
{incorrect_fields}       
Could you please review and issue an updated contract addressing the items above?

Let us know once the corrected version is ready for validation.

Regards,
OneCorp Support Team"""
    return output

email_body = """
Hi OneCorp Team,

Please find attached the Contract of Sale for the purchasers John & Jane Smith for:

Lot 95 – Fake Rise VIC 3336

Let us know if you need anything amended.

Regards,
Sarah
Contracts Manager
BuildWell Developments
"""
response = contract_validation_agent(email_body,"data/CONTRACT_OF_SALE_OF_REAL_ESTATE_V2.pdf")

In [15]:
import json

from openai import OpenAI
from datetime import datetime
from pydantic import BaseModel

class SigningAppointment(BaseModel):
    appointment_datetime: str   # "dd-mm-yyyy HH:MM"
    reminder_datetime: str      # "dd-mm-yyyy HH:MM"

def solicitor_approval_agent(email_body: str):
    # Extract appointment date and set reminder
    client = OpenAI()

    APPOINTMENT_EXTRACTOR_PROMPT = """
    You are a date-reasoning AI assistant. You will receive an email from a solicitor regarding a signing appointment.

    Your tasks:

    1. Identify the signing appointment date AND time from the email.
    - The date may be described naturally (e.g., “Thursday at 11:30am”).
    - Use the CURRENT LOCAL DATE: {current_date}
    - The user is in the Australia/Melbourne timezone (AEDT/AEST).
    - Convert the appointment into a full datetime in AEDT/AEST.
    - Format the final appointment datetime as: dd-mm-yyyy HH:MM.
    - If the email does not specify a time, default to 09:00 local time.

    2. Compute a reminder datetime:
    - appointment date + 2 days
    - time = 09:00 local time
    - Format as dd-mm-yyyy HH:MM.

    3. Return ONLY this JSON:

    {{
    "appointment_datetime_aedt": "<dd-mm-yyyy HH:MM>",
    "reminder_datetime_aedt": "<dd-mm-yyyy HH:MM>"
    }}

    Rules:
    - No UTC conversion — all dates must remain in Melbourne local time.
    - No seconds.
    - No markdown or explanations.
    - Output ONLY valid JSON.

    Here is the email:
    ---
    {email_body}
    ---
    """

    current_date = datetime.now().strftime("%d-%m-%Y")
    prompt = APPOINTMENT_EXTRACTOR_PROMPT.format(
        current_date=current_date,
        email_body=email_body
    )

    response = client.responses.parse(
        model="gpt-4.1",
        input=[
            {
                "role": "user",
                "content": [
                    {"type": "input_text", "text": prompt},
                ]
            }
        ],
        text_format=SigningAppointment  # Pydantic automatic validation!
    )

    response = json.loads(response.output_text)
    eoi_json = search_vector_store(email_body)
    
    response["Property_Address"] = eoi_json["Property_Address"]
    response["Purchaser"] = eoi_json["Purchaser"]

    filename = "deadlines/" + response["Property_Address"] + ".json"

    with open(filename, "w") as f:
        json.dump(response, f, indent=4)


    address = eoi_json["Property_Address"]
    purchasers = ""
    for i in eoi_json["Purchaser"]:
        purchasers += f"{i['First_Name']} {i['Last_Name']} & "
    purchasers = purchasers.rstrip(" & ")

    # Design Contract Release Email
    output = {}
    output["Email"] = f"""
Hi,

The solicitor has approved the contract for {purchasers}.
Could you please release the contract via DocuSign for purchaser signing?

Thanks,
OneCorp
"""
    output["Subject"] = f"Contract Request: {address}"
    return output

email_body = """
Hi Adrian,

We’ve completed our review of the contract for John & Jane Smith.
Everything is in order.

A signing appointment has been scheduled for Thursday at 11:30am with the clients.

Regards,
Michael Ken
Big Legal Firm
"""
print(solicitor_approval_agent(email_body))

NameError: name 'search_vector_store' is not defined

In [ ]:
import os
import datetime
import json
from datetime import datetime
from zoneinfo import ZoneInfo

def load_json(path):
    with open(path, "r") as f:
        return json.load(f)

def run_sla_check():
    """
    Runs daily at 9 AM AEDT via cron.
    For each JSON file, compare reminder date to today's date.
    If today's date matches, trigger an alert and delete the JSON file.
    """
    today = datetime.now(ZoneInfo("Australia/Melbourne")).strftime("%d-%m-%Y")
    DEADLINES_DIR = "deadlines"
    for file in os.listdir(DEADLINES_DIR):

        path = os.path.join(DEADLINES_DIR, file)
        data = load_json(path)
        reminder_full = data["reminder_datetime"]  
        reminder_date = reminder_full.split(" ")[0]  # Extract only dd-mm-yyyy

        if reminder_date == today:
            property_address = data.get("Property_Address")
            appointment_datetime = data.get("appointment_datetime")
            purchasers = ", ".join([f"{p['First_Name']} {p['Last_Name']}" for p in data.get("Purchaser")])

            subject = f"SLA Alert: Contract Not Signed by {reminder_date} for {purchasers} - {property_address}"
            email_body = f"""
Hi Team,

This is an automated SLA alert from the contract workflow.

The clients have not signed the Contract of Sale by the follow-up date.

Details:
- Purchasers: {purchasers}
- Property: {property_address}
- Signing Appointment: {appointment_datetime}
- Follow-up / SLA Date: {reminder_full}

Recommended actions:
1. Contact the solicitor to confirm the current signing status.
2. Follow up with the purchasers if needed.
3. Update the CRM / tracking system with any new information.

Please action this as soon as practical.

Regards,
OneCorp Contract Automation
support@onecorpaustralia.com.au
            """
            # sending email logic would go here
            print(f"🗑 Deleting: {file}")
            os.remove(path)


In [ ]:
import os
import json
from openai import OpenAI

client = OpenAI()

def end_of_contract_deletion_agent(email_body):
        DELETING_PROMPT = """
You are a Contract Completion Agent for OneCorp Australia.

Your job:
You will receive:
1) A DocuSign system email that mentions a contract signing event.
2) A list of JSON filenames currently in the deadlines directory.

ANY signing event (buyer only or fully executed) means we should stop tracking the deadline for that property.

Your task:
From the email, identify which JSON file in the list corresponds to the property mentioned in the email.

Important details about filenames:
- Filenames are the property address with ".json" at the end.
- They usually do NOT include the word "Lot" or the lot number.
- Example:
  - Email text: "Document: Contract of Sale – Lot 95 Fake Rise VIC 3336"
  - Filename: "Fake Rise VIC 3336.json"

Matching rules (follow these carefully):
1. Find the line in the email that starts with or contains "Document:" or clearly names the contract.
   Example: "Document: Contract of Sale – Lot 95 Fake Rise VIC 3336"

2. From that line, extract the **core property string** by:
   - Removing leading labels like "Document:", "Contract of Sale –", "Contract of Sale -"
   - Removing the "Lot <number>" prefix if present (e.g., "Lot 95 ")
   - Trimming spaces
   In the example above, the core property string becomes: "Fake Rise VIC 3336"

3. Normalise for comparison:
   - Lowercase everything
   - Ignore double spaces and punctuation like "–", "-", ":"
   - Do NOT include ".json" during comparison

4. For each filename in the list:
   - Remove the ".json" suffix
   - Lowercase the remaining text
   - Compare it to the core property string.
   - If the core property string appears to clearly match the filename text (same street, state, and postcode), select that filename.

5. If there is a clear, single best match, return that filename.
6. If there is no good match, or more than one plausible match, return null.

Output format (always JSON):

{
  "delete_filename": "<filename or null>"
}

Very important:
- Prefer an exact match on the full "Street Name + State + Postcode" segment.
- In this specific example:
  - Email document: "Contract of Sale – Lot 95 Fake Rise VIC 3336"
  - Filenames: "44 Pineview Crescent VIC 3977.json", "Fake Set VIC 3336.json", "12 Rivergum Road NSW 2259.json", "Fake Rise VIC 3336.json"
  - The correct result MUST be:
    {
      "delete_filename": "Fake Rise VIC 3336.json"
    }

Now, using these rules, process the given email and filename list.
        """

        DEADLINES_DIR = "deadlines"

        email_prompt = "DocuSign System Email is as follows: \n" + email_body + "\n\n"
        # Get filenames
        filenames = [
            f for f in os.listdir(DEADLINES_DIR)
            if f.endswith(".json")
        ]

        filenames_string = "Filenames are as follows: \n"
        # Convert to: "file1.json", "file2.json", "file3.json"
        filenames_string = filenames_string + ", ".join(f'"{name}"' for name in filenames)


        response = client.chat.completions.create(
            model="gpt-4.1-mini",   # or your preferred model
            messages=[
                {"role": "system", "content": DELETING_PROMPT},
                {"role": "user", "content": email_prompt + filenames_string}
            ],
            temperature=0.2
        )

        filename = json.loads(response.choices[0].message.content)["delete_filename"]
        if filename!= None:
            path = os.path.join(DEADLINES_DIR, filename)
            print(f"🗑 Deleting: {filename}")
            os.remove(path)
        else:
            print("No matching file to delete.")

email_body = """
The buyer has completed their signing.

Next step: Vendor signature required.

Document: Contract of Sale – Lot 95 Fake Rise VIC 3336

A final completed copy will be sent once all parties have signed.

— DocuSign System Notification
"""
end_of_contract_deletion_agent(email_body)

🗑 Deleting: Fake Rise VIC 3336.json


In [9]:
import requests

API_URL = "http://localhost:4000/send-email"

payload = {
    "recipient": "dr.prabhumane@gmail.com",
    "subject": "Contract Validation Complete",
    "body": "Hi team,\nAll contract fields match.\nProceed with solicitor."
}

res = requests.post(API_URL, json=payload)

print("Status:", res.status_code)
print("Response:", res.json())



Status: 200
Response: {'status': 'success', 'detail': 'Email sent successfully.'}


In [19]:
import requests


lvda = {

            "from_email": "dr.prabhumane@gmail.com",
            "to_email": "test@example.com",
            "subject": "EOI Signed: Lot 95 Fake Rise VIC 3336",
            "body": """Hi team,

The clients John & Jane Smith have signed the Expression of Interest for:

Lot 95, Fake Rise VIC 3336
Total Price: $550,000

EOI document is attached. Please proceed with the contract workflow.

Thanks,
Adrian
OneCorp
"""
        ,"attachments":["data/EOI_John_Jane-Smith.pdf"]    }

resp = requests.post("http://localhost:2000/incoming-email", json=lvda)
print(resp)

<Response [200]>


In [2]:
import requests


lvda = {

            "from_email": "dr.prabhumane@gmail.com",
            "to_email": "test@example.com",
            "subject": "Contract Request: Lot 95 Fake Rise VIC 33",
            "body": """Hi OneCorp Team,

Please find attached the Contract of Sale for the purchasers John & Jane Smith for:

Lot 95 – Fake Rise VIC 3336

Let us know if you need anything amended.

Regards,
Sarah
Contracts Manager
BuildWell Developments"""
        ,"attachments":["data/CONTRACT_OF_SALE_OF_REAL ESTATE_V1_test.pdf"]    }

resp = requests.post("http://localhost:2000/incoming-email", json=lvda)
print(resp)

<Response [200]>


In [31]:
import requests


lvda = {

            "from_email": "dr.prabhumane@gmail.com",
            "to_email": "test@example.com",
            "subject": "RE: Contract for Review – Smith – Lot 95 Fake Rise VIC 3336",
            "body": """Hi Adrian,

We’ve completed our review of the contract for John & Jane Smith.
Everything is in order.

A signing appointment has been scheduled for Thursday at 11:30am with the clients.

Regards,
Michael Ken
Big Legal Firm"""
        ,"attachments":[]    }

resp = requests.post("http://localhost:2000/incoming-email", json=lvda)
print(resp)

<Response [200]>


In [33]:
import requests


lvda = {

            "from_email": "dr.prabhumane@gmail.com",
            "to_email": "test@example.com",
            "subject": "Buyer Signed – Contract of Sale – Lot 95 Fake Rise VIC 3336",
            "body": """The buyer has completed their signing.

Next step: Vendor signature required.

Document: Contract of Sale – Lot 95 Fake Rise VIC 3336

A final completed copy will be sent once all parties have signed.

— DocuSign System Notification"""
        ,"attachments":[]    }

resp = requests.post("http://localhost:2000/incoming-email", json=lvda)
print(resp)

<Response [200]>


In [34]:
import requests


lvda = {

            "from_email": "dr.prabhumane@gmail.com",
            "to_email": "test@example.com",
            "subject": "Completed – Contract Fully Executed – Lot 95 Fake Rise VIC 3336",
            "body": """The envelope has been completed.
All parties have signed the document.

Download the final executed contract below:
[Download Executed Contract]

Document: Contract of Sale – Lot 95 Fake Rise VIC 3336

Thank you for using DocuSign.

— DocuSign System Notification"""
        ,"attachments":[]    }

resp = requests.post("http://localhost:2000/incoming-email", json=lvda)
print(resp)

<Response [200]>


In [1]:
from openai import OpenAI

client = OpenAI()

VECTOR_STORE_ID = "vs_693a5e43b3348191a934d5e4e3f8465c"

# List all files in the vector store
files = client.vector_stores.files.list(vector_store_id=VECTOR_STORE_ID)

# Delete each file
for file in files.data:
    client.vector_stores.files.delete(
        vector_store_id=VECTOR_STORE_ID,
        file_id=file.id
    )
    print("Deleted file:", file.id)

print("✔ All files deleted from vector store.")


Deleted file: file-AhMxErJaSpgLVdk8Uz3LDE
✔ All files deleted from vector store.
